# Data Splitting & Preprocessing
## Chia dữ liệu theo tỉ lệ 80-20 và tạo file .npz

Các bước thực hiện:
1. **Load danh sách file**: Lấy đường dẫn file từ raw_dataset
2. **Chia với stratify**: Train/Test (80/20) với phân bố đều các lớp
3. **Lưu file**: Copy file vào thư mục train/test

In [ ]:
# 1. Import thư viện cần thiết
import os
import json
import shutil
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [4]:
# 2. Định vị đường dẫn dataset
dataset_path = './raw_dataset'

# Lấy danh sách các lớp (thư mục con)
classes = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])

print(f"Tổng số lớp: {len(classes)}")
print(f"Các lớp: {classes}")

Tổng số lớp: 23
Các lớp: ['Clams', 'Corals', 'Crabs', 'Dolphin', 'Eel', 'Fish', 'Jelly Fish', 'Lobster', 'Nudibranchs', 'Octopus', 'Otter', 'Penguin', 'Puffers', 'Sea Rays', 'Sea Urchins', 'Seahorse', 'Seal', 'Sharks', 'Shrimp', 'Squid', 'Starfish', 'Turtle_Tortoise', 'Whale']


In [7]:
# 3. Lấy danh sách file và label
file_paths = []
labels = []

for class_idx, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    
    for img_file in images:
        img_path = os.path.join(class_path, img_file)
        file_paths.append(img_path)
        labels.append(class_idx)

print(f"Tổng số file: {len(file_paths)}")
print(f"Số lớp: {len(classes)}")

Tổng số file: 13711
Số lớp: 23


In [8]:
# 4. Chia dữ liệu với stratify (80% train / 20% test)
train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"\nChia dữ liệu với stratify:")
print(f"  • Train: {len(train_paths)} file ({len(train_paths)/len(file_paths)*100:.1f}%)")
print(f"  • Test:  {len(test_paths)} file ({len(test_paths)/len(file_paths)*100:.1f}%)")

# Tạo mapping class
class_mapping = {idx: class_name for idx, class_name in enumerate(classes)}


Chia dữ liệu với stratify:
  • Train: 10968 file (80.0%)
  • Test:  2743 file (20.0%)


In [9]:
# 5. Tổ chức file vào thư mục train/test
# Tạo cấu trúc thư mục
train_dir = 'split_data/train'
test_dir = 'split_data/test'

for class_name in classes:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# Copy file train
for src_path, label in tqdm(zip(train_paths, train_labels), total=len(train_paths), desc="Copying train"):
    class_name = class_mapping[label]
    filename = os.path.basename(src_path)
    dst_path = os.path.join(train_dir, class_name, filename)
    shutil.copy2(src_path, dst_path)

# Copy file test
for src_path, label in tqdm(zip(test_paths, test_labels), total=len(test_paths), desc="Copying test"):
    class_name = class_mapping[label]
    filename = os.path.basename(src_path)
    dst_path = os.path.join(test_dir, class_name, filename)
    shutil.copy2(src_path, dst_path)

print("Đã copy file vào thư mục train/test")

Copying test: 100%|██████████| 2743/2743 [00:01<00:00, 1843.64it/s]

Đã copy file vào thư mục train/test


In [10]:
# 6. Lưu thông tin chia dữ liệu
# Lưu class mapping
with open('split_data/class_mapping.json', 'w') as f:
    json.dump(class_mapping, f, indent=2, ensure_ascii=False)

# Lưu split info
split_info = {
    'total_files': len(file_paths),
    'train_files': len(train_paths),
    'test_files': len(test_paths),
    'train_ratio': 0.8,
    'test_ratio': 0.2,
    'stratified': True,
    'num_classes': len(classes)
}

with open('split_data/split_info.json', 'w') as f:
    json.dump(split_info, f, indent=2)

print("Đã lưu class_mapping.json và split_info.json")

Đã lưu class_mapping.json và split_info.json


In [12]:
# 7. Hoàn thành
print(f"\nCấu trúc thư mục 'split_data':")
print(f"   split_data/")
print(f"   ├── train/ (23 lớp)")
print(f"   ├── test/ (23 lớp)")
print(f"   ├── class_mapping.json")
print(f"   └── split_info.json")
print(f"\nTổng kết:")
print(f"   • Train: {len(train_paths)} ảnh (80%)")
print(f"   • Test:  {len(test_paths)} ảnh (20%)")
print(f"   • Chia với stratify để đảm bảo phân bố đều các lớp")


Cấu trúc thư mục 'split_data':
   split_data/
   ├── train/ (23 lớp)
   ├── test/ (23 lớp)
   ├── class_mapping.json
   └── split_info.json

Tổng kết:
   • Train: 10968 ảnh (80%)
   • Test:  2743 ảnh (20%)
   • Chia với stratify để đảm bảo phân bố đều các lớp
